In [15]:
import requests
import traceback
import json
import csv
import re 

In [16]:
# Build a simple csv file
# Data to be written to the CSV file
rows = [
    ["employee_name", "age", "city"],
    ["alice", "10.1", "new york"],
    ["bob", "11", "los angeles"],
    ["charlie", "12", "chicago"],
    ["john", "14.5", "soperton"],
    ["mike", "17.25", "dallas"]
]
rows

[['employee_name', 'age', 'city'],
 ['alice', '10.1', 'new york'],
 ['bob', '11', 'los angeles'],
 ['charlie', '12', 'chicago'],
 ['john', '14.5', 'soperton'],
 ['mike', '17.25', 'dallas']]

In [17]:
# Write the data file
file_path = '/home/ubuntu/ragllm2/data/employee_data.csv'

# Open a file in write mode
with open(file_path, 'w', newline='') as file:
    writer = csv.writer(file)
    
    # Writing rows
    for row in rows:
        writer.writerow(row)

In [18]:
# Now let reprocess the data files
import requests

# Define the URL to which the form submits data
# Assuming 'base_url' is the domain where the form resides, you need to replace it with the actual domain
base_url = 'http://localhost:5000'
endpoint = '/reprocess'
url = base_url + endpoint

# Make the POST request to the endpoint
response = requests.post(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful... Context updated")
else:
    print(f"Request failed. Status code: {response.status_code}")


Request successful... Context updated


In [19]:
# Now lets query the model
# The URL to which the request is being sent
url = 'http://localhost:5000/query'

# Headers to be sent with the request
headers = {
    'Content-Type': 'application/json'
}

# Data to be sent in the request
data = {
    'query': 'What is Johns age?'
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check the response
if response.status_code == 200:
    print("Success:", response.text)
else:
    print("Error:", response.status_code, response.text)

Success: {"qhid":"53b4e62c-75fd-41fe-8ffb-e08a39a57902","response":"14.5"}



In [20]:
# Get the query history id - qhid
qhid = response.json()['qhid']
qhid

'53b4e62c-75fd-41fe-8ffb-e08a39a57902'

In [21]:
# Now lets cointinue the discussion using the quid
# The URL to which the request is being sent
url = 'http://localhost:5000/query'

# Headers to be sent with the request
headers = {
    'Content-Type': 'application/json'
}

# Data to be sent in the request
data = {
    'query': 'What city does he live in?',
    'qhid': qhid
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check the response
if response.status_code == 200:
    print("Success:", response.text)
else:
    print("Error:", response.status_code, response.text)

Success: {"qhid":"53b4e62c-75fd-41fe-8ffb-e08a39a57902","response":" Soperton"}



In [22]:
# Now lets plot some data
# The URL to which the request is being sent
url = 'http://localhost:5000/query'

# Headers to be sent with the request
headers = {
    'Content-Type': 'application/json'
}

# Data to be sent in the request
data = {
    'query': 'Generate python code to plot a historgram of the employees by age.',
    'qhid': qhid
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(data))

# Check the response
if response.status_code == 200:
    print("Success:", response.text)
else:
    print("Error:", response.status_code, response.text)

Success: {"qhid":"53b4e62c-75fd-41fe-8ffb-e08a39a57902","response":"\n```python\nimport matplotlib.pyplot as plt\nimport numpy as np\nfrom scipy import stats\n\n# Data = [{'name': 'John', 'age': 14.5, 'city': 'Soperton'},\n          {'name': 'Bob', 'age': 11, 'city': 'Los Angeles'},\n          {'name: 'Charlie', 'age': 12, 'city': 'Chicago'},\n          {'name': 'Alice', 'age': 10.1, 'city': 'New York'}]\n\n# Calculate_Age = [stats.moment(Data[i]['age'])[0] for i in range(len(Data))]\n\n Age_Histogram = np.histogram(Calculate_Age, bins=np.arange(8-12), density=True)\n \nplt.figure(figsize=(15,6))\nplt.bar(*Age_Histogram[0], heights=Age_Histogram[1])\nplt.title('Employee Age Distribution')\nplt.xlabel('Age Range')\nplt.y"}



In [23]:
# Convert to json data
response_data = json.loads(response.text)

# Extracting the code string
code_string = response_data['response'].split("```python\n", 1)[1].rsplit("\n```", 1)[0]

# Function to correct the code
def correct_code(code):
    corrected_lines = []
    for line in code.split('\n'):
        # Detect lines that might be comments but are not marked as such
        if re.match(r"^[A-Z].*", line) and not re.match(r"^(import |from |class |def |#)", line):
            line = "# " + line
        corrected_lines.append(line)
    
    return '\n'.join(corrected_lines)

# Apply the correction
corrected_code = correct_code(code_string)
print(corrected_code)

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

# Data = [{'name': 'John', 'age': 14.5, 'city': 'Soperton'},
          {'name': 'Bob', 'age': 11, 'city': 'Los Angeles'},
          {'name: 'Charlie', 'age': 12, 'city': 'Chicago'},
          {'name': 'Alice', 'age': 10.1, 'city': 'New York'}]

# Calculate_Age = [stats.moment(Data[i]['age'])[0] for i in range(len(Data))]

 Age_Histogram = np.histogram(Calculate_Age, bins=np.arange(8-12), density=True)
 
plt.figure(figsize=(15,6))
plt.bar(*Age_Histogram[0], heights=Age_Histogram[1])
plt.title('Employee Age Distribution')
plt.xlabel('Age Range')
plt.y


In [26]:
corrected_code = """import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

Data = [{'name': 'John', 'age': 14.5, 'city': 'Soperton'},
          {'name': 'Bob', 'age': 11, 'city': 'Los Angeles'},
          {'name': 'Charlie', 'age': 12, 'city': 'Chicago'},
          {'name': 'Alice', 'age': 10.1, 'city': 'New York'}]

Calculate_Age = [stats.moment(Data[i]['age'])[0] for i in range(len(Data))]

Age_Histogram = np.histogram(Calculate_Age, bins=np.arange(8-12), density=True)
 
plt.figure(figsize=(15,6))
plt.bar(*Age_Histogram[0], heights=Age_Histogram[1])
plt.title('Employee Age Distribution')
plt.xlabel('Age Range')
plt.y"""

In [27]:
exec(corrected_code)

IndexError: invalid index to scalar variable.